# Проект по SQL "Анализ базы данных сервиса для чтения книг по подписке"

Проведен анализ базы данных сервиса для чтения книг по подписке. Посчитано количество книг, вышедших после 1 января 2000 года. Для каждой книги определены количество обзоров и средняя оценка. Определено издательство, которое выпустило наибольшее число книг толще 50 страниц — исключены из анализа брошюры. Определен автора с самой высокой средней оценкой книг — среди книг с 50 и более оценками. Посчитано среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

#### Цель 
проанализировать базу данных о книгах, издательствах, авторах, а также пользовательские обзоры книг для формулирования ценностного предложения для нового продукта.  

#### Задания
<a href="#раздел1">Посчитать сколько книг вышло после 1 января 2000 года;</a>  
<a href="#раздел2">Для каждой книги посчитать количество обзоров и среднюю оценку;</a>  
<a href="#раздел3">Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр);</a>  
<a href="#раздел4">Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;</a>  
<a href="#раздел5">Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.</a>  

#### План выполнения задания
Описать цели исследования;  
Исследовать таблицы — выведите первые строки;  
Сделать по одному SQL-запросу для решения каждого задания;  
Вывести результаты каждого запроса в тетрадке;  
Описать выводы по каждой из решённых задач.  

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


# Исследовать таблицы — выведите первые строки

In [2]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# проверяем типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,table_name,column_name,data_type,is_nullable
0,books,book_id,integer,NO
1,ratings,rating,integer,YES
2,reviews,review_id,integer,NO
3,reviews,book_id,integer,YES
4,books,num_pages,integer,YES
5,authors,author_id,integer,NO
6,books,publication_date,date,YES
7,books,publisher_id,integer,YES
8,publishers,publisher_id,integer,NO
9,books,author_id,integer,YES


<a id="раздел1"></a>

# Задачи
**Задание 1: Посчитать количество книг, вышедших с 1 января 2000 года**

In [8]:
cnt_books = '''SELECT COUNT(book_id) 
FROM books 
WHERE publication_date::date >= '2000-01-01' '''
con=engine.connect()
cnt_books = pd.io.sql.read_sql(sql=text(cnt_books), con = con)
display(cnt_books)

,count
0,821


Количество книг, вышедших с 1 января 2000 года, составляет 821.

<a id="раздел2"></a>

**Задание 2: Для каждой книги посчитать количество обзоров и среднюю оценку**

In [9]:
about_books = '''
SELECT title, COUNT(DISTINCT(review_id)) as cnt_re, COALESCE(ROUND(AVG(rating), 1), 0) as avg_ra

FROM books as b LEFT JOIN reviews as re on b.book_id = re.book_id LEFT JOIN ratings as ra on b.book_id = re.book_id
GROUP BY b.book_id
ORDER BY cnt_re DESC, avg_ra
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(about_books), con = con)

,title,cnt_re,avg_ra
0,Twilight (Twilight #1),7,3.9
1,Water for Elephants,6,3.9
2,The Glass Castle,6,3.9
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,3.9
4,The Curious Incident of the Dog in the Night-Time,6,3.9
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,0.0
996,The Natural Way to Draw,0,0.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,0.0
998,Essential Tales and Poems,0,0.0


Из 1000 книг в базе популярные книги имеют по 6 обзоров и одна 7 оброзов, у некоторых книг не имеются обзоры и средняя оценка

<a id="раздел3"></a>

**Задание 3: Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключим из анализа брошюры**

In [10]:
query = '''SELECT publisher, COUNT(b.book_id) as cnt_books
FROM publishers as p INNER JOIN books as b on p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY cnt_books DESC
LIMIT 1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,cnt_books
0,Penguin Books,42


 Издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books с 42 книгами

<a id="раздел4"></a>

**Задание 4: Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [11]:
query = '''SELECT DISTINCT a.author as au, AVG(ra.rating) as avg_ra, COUNT(ra.rating_id) as cnt_ra
FROM authors as a INNER JOIN books as b on b.author_id = a.author_id INNER JOIN ratings as ra on ra.book_id = b.book_id
WHERE b.book_id in (SELECT book_id FROM ratings GROUP BY book_id HAVING COUNT(rating) > 50)
GROUP BY author
ORDER BY avg_ra DESC
LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,au,avg_ra,cnt_ra
0,J.K. Rowling/Mary GrandPré,4.287097,310


Автором с самой высокой средней оценкой книг, у которых 50 и более оценок, является Джоан Роулинг.

<a id="раздел5"></a>

**Задание 5: Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [12]:
query = '''
WITH cnt_ra as 
(SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 48)
SELECT AVG(cnt_re) as avg_re FROM 
    (SELECT COUNT(review_id) as cnt_re FROM reviews WHERE username in
            (SELECT username FROM cnt_ra)
    GROUP BY username) as f1
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_re
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составялет 24%.

# Выводы по решенным задачам:

Количество книг, вышедших после 1 января 2000 года, составляет 819.  
Из 1000 книг в базе популярные книги имеют по 6 обзоров и одна 7 оброзов, у некоторых книг не имеются обзоры и средняя оценка. Издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books с 42 книгами.  Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составялет 24%.
В отношении книг, у которых мало или вовсе нет обзоров и отсутствует средняя оценка, можем порекомендовать мотивировать пользователей на проведение обзоров и оценивание книг (бонусные баллы, демо-версии книг бесплатно и т.д.). Также можно провести работу с издательствами, которые выпустили мало книг.